In [1]:
import sys, os
sys.path.append(os.path.dirname(os.getcwd()))

from problem_generator import Problem_Generate_Model
from langchain.schema import HumanMessage, SystemMessage
from util import random_word
from util import find_random_question
import re
import random

In [5]:
total_cost = 0
default_model = "gpt-4o-mini"
verbose = True
models = [default_model]
problem_generate_model = Problem_Generate_Model(
    models, use_cache=True, temperature=1, verbose=verbose, api_key_path="./../API_KEY/llm_api_key.json"
)
main_title = random_word()
if verbose:
    print(main_title)

problem_generate_model.request_models_responses(
    [
        SystemMessage(content="주제어와 관련된 일상적인 단어 1개를 제시하라."),
        HumanMessage(content="주제어: " + main_title + " 단어: "),
    ]
)
word_responses = problem_generate_model.get_model_responses()
word = word_responses[0][1]
def clean_text(text):
            pattern = r':\s*(.*)$'
            match = re.search(pattern, text)
            if match:
                return match.group(1).strip()
            text = re.sub(r'\n', ' ', text)  # 개행 문자를 공백으로 대체
            return text.strip()
word = clean_text(word)
total_cost += float(re.search(r'Total Cost \(KRW\): ₩([\d.]+)', word_responses[0][2]).group(1))

keyword = 'TOPIK_2_READING'
problem_type = 'READING_2_PROBLEM_TYPE_6'
random_question = find_random_question(keyword, problem_type, detail_type=1, count=2, api_key_path="./../API_KEY/topik_api_key.json")
if verbose:
    print(random_question)

def replace_placeholder(sentence, new_phrase):
    pattern = r'\(\s*\)'
    replaced_sentence = re.sub(pattern, f'({new_phrase})', sentence)
    return replaced_sentence

example_dict = []
for item in random_question:
    example_text = replace_placeholder(item['example'], item['answer'])
    problem_generate_model.request_models_responses([
    SystemMessage(content="문장의 주제 단어 1개를 제시하라."),
    HumanMessage(content="문장: "+ example_text + "주제 단어: "),
        ]
    )
    total_cost += float(re.search(r'Total Cost \(KRW\): ₩([\d.]+)', problem_generate_model.get_model_responses()[0][2]).group(1))
    title_responses = problem_generate_model.get_model_responses()[0][1]
    temp = {"주제": title_responses, "예문": example_text, "선택지": ', '.join(item['selector']), "정답": item['answer']}
    example_dict.append(temp)
if verbose:
    print(example_dict)

example_prompt = ""
for item in example_dict:
    example_str = f"주제: {item['주제']}, 예문: {item['예문']}\n 선택지: {item['선택지']}\n 정답: {item['정답']}"
    example_prompt += example_str + "\n"
if verbose:
    print(example_prompt)
problem_generate_model.request_models_responses(
    [
        SystemMessage(
            content="#예시처럼 주어지는 주제어에 대한 빈칸에 들어갈 알맞은 내용 고르기 문제 1개를 생성해줘. #답변 예시 "
            + example_prompt
        ),
        HumanMessage(content="주제: " + word),
    ]
)
problem_responses = problem_generate_model.get_model_responses()
total_cost += float(re.search(r'Total Cost \(KRW\): ₩([\d.]+)', problem_responses[0][2]).group(1))
if verbose:
    print(problem_responses[0][1])

def clean_text(text):
    pattern = r':\s*(.*)$'
    match = re.search(pattern, text)
    if match:
        return match.group(1).strip()
    text = re.sub(r'\n', ' ', text)  # 개행 문자를 공백으로 대체
    return text.strip()

def extract_parts(text):
    # 예문 추출
    example_pattern = r'예문:\s*(.*)'
    example_match = re.search(example_pattern, text)
    example = example_match.group(1).strip() if example_match else None

    # 선택지 추출
    options_pattern = r'선택지:\s*(.*)'
    options_match = re.search(options_pattern, text)
    if options_match:
        options = options_match.group(1).strip().split(', ')
        for option in options:
            option = clean_text(option)
    else:
        options = []

    # 정답 추출
    answer_pattern = r'정답:\s*(.*)'
    answer_match = re.search(answer_pattern, text)
    answer = answer_match.group(1).strip() if answer_match else None

    return example, options, answer

example, options, answer = extract_parts(problem_responses[0][1])

def remove_parentheses(sentence):
    sentence = re.sub(r'\(', '', sentence)
    sentence = re.sub(r'\)', '', sentence)
    return sentence

example = remove_parentheses(example)
random.shuffle(options)
result = {
    "example": example.replace(answer, "(  )", 1),
    "selector": options,
    "answer": answer,
    "eval_answer": answer,
    "eval_explain": example,
    "total_cost": total_cost
}
print(result)

    

하나
gpt-4o-mini | 
통합

OpenAI Tokens Used: 42
    Prompt Tokens: 40
    Completion Tokens: 2
Successful Requests: 1
Total Cost (USD): $3e-05
Total Cost (KRW): ₩0.0405
--------------------------------------------------------------------------------------------------
[{'example': '여권은 국제 신분증으로 통할 뿐만 아니라 한 국가의 정체성과 문화를 드러내기도 한다. 그래서 많은 국가들은 여권의  ( ) 결정 한다. 여권 표지의 색깔을 국가의 상징색으로 표현하거나 국가를 상징하는 문양을 넣기도 한다. 또한 여권에 불빛이 비치면 주요 문화재가 화려하게 보이도록 한 것도 있다.', 'selector': ['두께를 자율적으로', '디자인을 신중하게', '이용 범위를 확실히', '발급 방식을 구체적으로'], 'answer': '디자인을 신중하게'}, {'example': '필름형 스피커는 종이보다 얇고 투명해서 ( ) 사용할 수 있다. 이 스피커를 활용하면 컴퓨터 화면이나 액자뿐만 아니라 벽이나 천장에서도 소리가 나오게 할 수 있다. 또 소재가 부드러워서 옷이나 커튼에 달아도 불편함 없이 사용이 가능하다.', 'selector': ['소리 크기를 높여', '단독 형태로 분리해', '다양한 물건에 붙여', '특수한 영역을 제한해'], 'answer': '다양한 물건에 붙여'}]
gpt-4o-mini | 
여권

OpenAI Tokens Used: 135
    Prompt Tokens: 133
    Completion Tokens: 2
Successful Requests: 1
Total Cost (USD): $0.0001
Total Cost (KRW): ₩0.135
---------------------------------------------------